In [7]:
from os import path
import struct
import numpy as np


In [23]:
def load_mnist():
#     print(path.abspath('load_data.ipynb')) # 获取绝对路径
#     label_path = os.path.join(path,'%s-labels-idx1-ubyte' %kind)
#     image_path = os.path.join(path,'%s-images-idx3-ubyte' %kind)
    with open('train-labels.idx1-ubyte',"rb") as lbfile:
        magic, n = struct.unpack('>II', lbfile.read(8))
        labels = np.fromfile(lbfile, dtype=np.uint8)
    with open('train-images.idx3-ubyte','rb') as imfile:
        magic, n, row, clom = struct.unpack('>IIII',imfile.read(16))
        images = np.fromfile(imfile,dtype=np.uint8).reshape(len(labels),784)  # 每张图片为28*28的size
    return images, labels
train_image, train_label = load_mnist()

In [28]:
print(train_image.shape)
print(train_label.shape)

(60000, 784)
(60000,)


In [48]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D
from keras.optimizers import SGD

In [38]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [32]:
(image_train, label_train),(image_test, label_test) = mnist.load_data()

11337728/11490434 [============================>.] - ETA: 0s

In [51]:
print(type(image_train),type(label_train))
print(image_train.shape)
print(image_test.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [39]:
image_train = image_train.reshape(60000,28,28,1)    # 以tensorflow为后端时，通道放在最后一维
image_test = image_test.reshape(10000,28,28,1)
label_train = to_categorical(lab el_train, num_classes=10)    # one-hot编码为十类
label_test = to_categorical(label_test, num_classes=10)

## 模拟LeNet-5
### LeNet-5特点
* 输入图片shape为32x32x1，不加输入层，总共为7层结构，其中两层卷积两层池化嵌套，再加三层全连接层，最后一个全连接层为输出。
### 实现疑问
* LeNet-5的模型输入层为32x32大小，可为什么MNIST数据集的每张图片大小是28x28？

In [60]:
# create model
lenet = Sequential()
print("input size:",image_train.shape)
L1 = Conv2D(filters=6,kernel_size=(5,5),padding='valid',input_shape=(28,28,1),activation='tanh')
lenet.add(L1)  # 6*24*24
print("layer one shape:",L1.output_shape)
L2 = MaxPooling2D()
lenet.add(L2)  #默认的pool_size为（2,2）
print("layer tow shape:",L2.output_shape)
L3 = Conv2D(16,(5,5),padding='valid',activation='tanh')
lenet.add(L3)
L4 = MaxPooling2D()
lenet.add(L4)    # 16个4*4的矩阵
lenet.add(Flatten())    # 将矩阵压平为一维
print("layer three shape:",L3.output_shape)
print("layer four shape:",L4.output_shape)
lenet.add(Dense(120,activation='tanh'))
lenet.add(Dense(84,activation='tanh'))
lenet.add(Dense(10,activation='softmax'))
# compile model
lenet.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.1),metrics=['accuracy'])
# train model
lenet.fit(image_train,label_train,batch_size=128,epochs=2)
# 评估模型
score = lenet.evaluate(image_test,label_test)
print("\n")
print("total loss on testing set:", score[0])
print("accuracy of testing set:", score[1])

input size: (60000, 28, 28, 1)
layer one shape: (None, 24, 24, 6)
layer tow shape: (None, 12, 12, 6)
layer three shape: (None, 8, 8, 16)
layer four shape: (None, 4, 4, 16)
Epoch 1/2
60000/60000 [==============================] - 26s - loss: 0.3271 - acc: 0.9056    
Epoch 2/2
 9920/10000 [============================>.] - ETA: 0s

total loss on testing set: 0.110166930688
accuracy of testing set: 0.9648
